In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

In [2]:

data = pd.read_csv("test_assignment_sim.csv")
site_co = pd.read_csv("site_coordinates.csv")

sites_datas = []
for i in range(49):
    slice = data.iloc[:,[0,1,2,3]].copy()
    slice['thickness'] = data.iloc[:,i+4]
    slice['SITE'] = i # in lgb, catagory should be a int
    slice['S_X'] = site_co['SITE_X'][i]
    slice['S_Y'] = site_co['SITE_Y'][i]
    sites_datas.append(slice)

site_thickness = sites_datas[0]
for j in range(1,49):
    site_thickness = pd.concat([site_thickness,sites_datas[j]])

site_thickness.reset_index(drop=True)
site_thickness['TOOL'] = site_thickness['TOOL'].astype('category')
site_thickness.describe()
thickness_dmy = pd.get_dummies(site_thickness)

clist = list(thickness_dmy.columns)
clist.pop(4)
clist.append(clist.pop(3))

thickness_dmy = thickness_dmy[clist]
thickness_dmy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FLOWFACTOR  24500 non-null  float64
 1   SPACING     24500 non-null  float64
 2   DEP TIME    24500 non-null  float64
 3   S_X         24500 non-null  float64
 4   S_Y         24500 non-null  float64
 5   TOOL_1      24500 non-null  uint8  
 6   TOOL_2      24500 non-null  uint8  
 7   TOOL_3      24500 non-null  uint8  
 8   TOOL_4      24500 non-null  uint8  
 9   thickness   24500 non-null  float64
dtypes: float64(6), uint8(4)
memory usage: 1.4 MB


In [3]:
dftrain,dftest = train_test_split(thickness_dmy,test_size=0.1)

#normallize data
mean = dftrain.mean()
std = dftrain.std()
train_data = (dftrain - mean) / std
train_data=train_data.astype('float32')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22050 entries, 45 to 340
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FLOWFACTOR  22050 non-null  float32
 1   SPACING     22050 non-null  float32
 2   DEP TIME    22050 non-null  float32
 3   S_X         22050 non-null  float32
 4   S_Y         22050 non-null  float32
 5   TOOL_1      22050 non-null  float32
 6   TOOL_2      22050 non-null  float32
 7   TOOL_3      22050 non-null  float32
 8   TOOL_4      22050 non-null  float32
 9   thickness   22050 non-null  float32
dtypes: float32(10)
memory usage: 1.0 MB


In [4]:

BATCH_SIZE = 30
noise_dim = 3

ts_data = tf.data.Dataset.from_tensor_slices(train_data.values).shuffle(len(dftrain)).batch(BATCH_SIZE)


def make_generator_model():
    model = keras.Sequential()
    model.add(layers.Dense(18, use_bias=False, input_shape=(12,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(36))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(36))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(18))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1))
    return model

def make_discriminator_model():
    model = keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=(10,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(40))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(20))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1))
    return model


generator = make_generator_model()
discriminator = make_discriminator_model()
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(real_data):
    noise = tf.random.normal([BATCH_SIZE, noise_dim]) # 30 by 3
    features = tf.slice(real_data,[0,0],[BATCH_SIZE,9]) # cut thickness out, features are 30 by 9
    thickness = tf.slice(real_data,[0,9],[BATCH_SIZE,1]) #thickness are 30 by 1
    noise_and_features = tf.concat([noise,features],1) # noise + features = 30 by 12

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:  # tape, helping gradient calculation
        generated_thickness = generator(noise_and_features, training=True) # 30 by 1
        gethickness_and_features = tf.concat([features,generated_thickness],1) # 30 by 10

        real_output = discriminator(real_data, training=True) # 30 by 1  , forward 
        fake_output = discriminator(gethickness_and_features, training=True) #30 by 1 , forward

        gen_loss = generator_loss(fake_output) # if fake_output is 1, generator will not update net. if 0, update
        disc_loss = discriminator_loss(real_output, fake_output) # learn from a real one, and a fake one. 

    # get gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # update
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, epochs):
    
    if(os.path.isfile("GAN-1.h5")):
        os.remove('GAN-1.h5')
    

    for epoch in range(epochs):
        start = time.time()

        for data_batch in dataset:
            train_step(data_batch)
        
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    generator.save('GAN-1.h5')

In [5]:
train(ts_data,500) #1000

Time for epoch 1 is 10.944549322128296 sec
Time for epoch 2 is 6.09403920173645 sec
Time for epoch 3 is 6.07432222366333 sec
Time for epoch 4 is 6.305443048477173 sec
Time for epoch 5 is 6.2270519733428955 sec
Time for epoch 6 is 6.101850748062134 sec
Time for epoch 7 is 6.59287691116333 sec
Time for epoch 8 is 6.264510154724121 sec
Time for epoch 9 is 6.410682201385498 sec
Time for epoch 10 is 6.408581972122192 sec
Time for epoch 11 is 6.167527914047241 sec
Time for epoch 12 is 6.502094984054565 sec
Time for epoch 13 is 6.136268854141235 sec
Time for epoch 14 is 6.246300220489502 sec
Time for epoch 15 is 6.128796815872192 sec
Time for epoch 16 is 6.221546173095703 sec
Time for epoch 17 is 6.192618131637573 sec
Time for epoch 18 is 6.122527837753296 sec
Time for epoch 19 is 6.172865867614746 sec
Time for epoch 20 is 6.355793237686157 sec
Time for epoch 21 is 6.202919006347656 sec
Time for epoch 22 is 6.098844766616821 sec
Time for epoch 23 is 6.113028049468994 sec
Time for epoch 24 is 

Time for epoch 190 is 6.09248423576355 sec
Time for epoch 191 is 6.097610235214233 sec
Time for epoch 192 is 6.110742092132568 sec
Time for epoch 193 is 6.114875078201294 sec
Time for epoch 194 is 6.106396198272705 sec
Time for epoch 195 is 6.077710151672363 sec
Time for epoch 196 is 6.113242864608765 sec
Time for epoch 197 is 6.077045917510986 sec
Time for epoch 198 is 6.102153062820435 sec
Time for epoch 199 is 6.109323024749756 sec
Time for epoch 200 is 6.099867105484009 sec
Time for epoch 201 is 6.083092927932739 sec
Time for epoch 202 is 6.12821102142334 sec
Time for epoch 203 is 6.087102890014648 sec
Time for epoch 204 is 6.148722171783447 sec
Time for epoch 205 is 6.114151954650879 sec
Time for epoch 206 is 6.1210949420928955 sec
Time for epoch 207 is 6.218916893005371 sec
Time for epoch 208 is 6.278964996337891 sec
Time for epoch 209 is 6.108535051345825 sec
Time for epoch 210 is 6.132256031036377 sec
Time for epoch 211 is 6.0898988246917725 sec
Time for epoch 212 is 6.28012990

Time for epoch 376 is 6.072101831436157 sec
Time for epoch 377 is 6.096612215042114 sec
Time for epoch 378 is 6.117962121963501 sec
Time for epoch 379 is 6.125623941421509 sec
Time for epoch 380 is 6.325892210006714 sec
Time for epoch 381 is 6.316090822219849 sec
Time for epoch 382 is 6.1133880615234375 sec
Time for epoch 383 is 6.111058950424194 sec
Time for epoch 384 is 6.117099046707153 sec
Time for epoch 385 is 6.164247989654541 sec
Time for epoch 386 is 6.190617084503174 sec
Time for epoch 387 is 6.1440980434417725 sec
Time for epoch 388 is 6.114311218261719 sec
Time for epoch 389 is 6.132784128189087 sec
Time for epoch 390 is 6.136739253997803 sec
Time for epoch 391 is 6.118120908737183 sec
Time for epoch 392 is 6.1718809604644775 sec
Time for epoch 393 is 6.057010889053345 sec
Time for epoch 394 is 6.369075298309326 sec
Time for epoch 395 is 6.126878976821899 sec
Time for epoch 396 is 6.099164009094238 sec
Time for epoch 397 is 6.110263824462891 sec
Time for epoch 398 is 6.11946

In [6]:


ganmodel = keras.models.load_model('GAN-1.h5',compile=False)

y_test = np.array(dftest['thickness'])

test_data = (dftest - mean) / std
testnoise = pd.DataFrame(np.random.randn(len(y_test), 3), columns=['n_1','n_2','n_3']) # add three columns of noise
noise_testdata = pd.concat([testnoise,test_data.reset_index(drop=True)],axis=1)

# testing data prediction
y_pred = ganmodel.predict(np.array(noise_testdata.drop('thickness', axis='columns')))
# reversing-normalization of y_pred
y_pred = np.reshape(y_pred * std['thickness'] + mean['thickness'], y_test.shape)

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 16.62806224779175
